# Requirements

In [ ]:
#!pip install espeakng
#!pip install speechbrain
#!pip install -r requirements_1.txt
#import nltk
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
#!pip install numpy==1.24 

In [ ]:
#!pip install python-Levenshtein
#!pip install num2words



# Imports

In [ ]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Auto reload imports
%load_ext autoreload
%autoreload 2

In [ ]:
# Add user class to path
import sys
sys.path.append('../reader')
sys.path.append('../shared')

In [ ]:
from copy import copy
import os
from num2words import num2words

from Levenshtein import distance
import os
import librosa   
import soundfile as sf


In [ ]:
import IPython
from TTS.api import TTS
from color import Color

In [ ]:

def expose(prefix:str=""):
    import torch
    print (f"{prefix}{torch.__version__ = }")
    import transformers
    print (f"{prefix}{transformers.__version__ = }")
    import nltk
    print (f"{prefix}{nltk.__version__ = }")
    import speechbrain
    print (f"{prefix}{speechbrain.__version__ = }")
    import TTS
    print (f"{prefix}{TTS.__version__ = }")
    import datasets
    print (f"{prefix}{datasets.__version__ = }")
expose()


# Define inputs

In [ ]:
texts = [
    """"""
        ]

# SST Model

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset

In [ ]:


# load model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-large-v2")
stt_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v2")
stt_model.config.forced_decoder_ids = None

In [ ]:
stt_model.cuda()

In [ ]:
def transcribe(model, wav, samplerate=24000):
    input_features = processor(wav, sampling_rate=samplerate, return_tensors="pt").input_features 
    input_features = input_features.to("cuda")
    predicted_ids = model.generate(input_features)
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
    return transcription

# XTTS

In [ ]:
!nvcc --version

In [ ]:
#!pip install deepspeed


In [ ]:
from TTS.tts.models.xtts import Xtts
from TTS.utils.manage import ModelManager
from TTS.tts.configs.xtts_config import XttsConfig

In [ ]:
#ModelManager().list_models()

In [ ]:
model_name = "tts_models/multilingual/multi-dataset/xtts_v1.1"
ModelManager().download_model(model_name)

In [ ]:
ModelManager().download_model(model_name)

In [ ]:
#!ls /root/.local/share/tts/tts_models--multilingual--multi-dataset--xtts_v1.1

In [ ]:
model_path = "/root/.local/share/tts/tts_models--multilingual--multi-dataset--xtts_v1.1/model.pth"
model_path

In [ ]:
config_path = "/root/.local/share/tts/tts_models--multilingual--multi-dataset--xtts_v1.1/config.json"
vocab_path = "/root/.local/share/tts/tts_models--multilingual--multi-dataset--xtts_v1.1/vocab.json"

In [ ]:
config = XttsConfig()
config.load_json(config_path)

config.keys()

In [ ]:

model = Xtts.init_from_config(config)
model.load_checkpoint(
    config,
    checkpoint_path=model_path,
    vocab_path=vocab_path,
    eval=True,
    use_deepspeed=False
)

In [ ]:

model.cuda()

In [ ]:

outputs = model.synthesize(
    texts[1],
    config,
    speaker_wav="../data/trey.wav",
    gpt_cond_len=3,
    language="en",
    top_k=3,
    top_p=5,
    decoder_iterations=10,
    #repetition_penalty=.75
)

In [ ]:
outputs.keys()

In [ ]:
wav = outputs["wav"]

In [ ]:
#text = "one nine five five zero"
#text = "zero nine zero twenty seven"
text = "Alec you get a big bonus!"
synth_params = {
    "gpt_cond_len" : 8,
    "top_k" : 50, # Default 50
    "top_p" : 0.85, # Default 0.85
    "decoder_iterations" : 10, # Default 30
    "temperature" : 0.1, # Default 0.85
    "length_penalty" : 1.0, # Default 1.0
    "repetition_penalty" : 2.0, # Default 2.0
    "cond_free_k" : 2.0, # Default 2.0
    "diffusion_temperature" : 1.0, # Default 1.0    
}
outputs = model.synthesize(
    text,
    config,
    speaker_wav="../data/trey.wav",
    language="en",
    **synth_params
    
)

In [ ]:
config

In [ ]:
IPython.display.display(IPython.display.Audio(outputs["wav"], rate=24000, autoplay=False))

In [ ]:
# Write to local file
sf.write("cache/cache_24.wav", outputs["wav"], 24000)

In [ ]:
os.system(f"ffmpeg -y -i cache/cache_24.wav -ar 16000 cache/cache_16.wav")

In [ ]:
def transcribe_output(wav):
    sf.write("cache/cache_24.wav", wav, 24000)
    os.system(f"ffmpeg -y -hide_banner -loglevel error -i cache/cache_24.wav -ar 16000 cache/cache_16.wav")
    y, fs = sf.read("cache/cache_16.wav")
    return transcribe(stt_model, y, samplerate=16000)
    

## Param tune

In [ ]:

lthresh_good = 10
lthresh_mid = 20
lthresh_bad = 30

In [ ]:
def get_lcolor(ldist):
    lcolor = Color.F_Green
    if ldist > lthresh_mid:
        lcolor = Color.F_Yellow
    if ldist > lthresh_bad:
        lcolor = Color.F_Red
    return lcolor

### Gpt conds

In [ ]:
gpt_conds = [1,3,5,7,11,15,21,32,64,128,256,512,1024,2048,4096]
for gpt_cond in gpt_conds:
    text = ""
    
    test_config = copy(synth_params)
    test_config["gpt_cond_len"] = gpt_cond
    
    outputs = model.synthesize(
        text,
        config,
        speaker_wav="../data/trey.wav",
        language="en",
        **test_config
        #repetition_penalty=.75
    )
    transcription = transcribe_output(outputs["wav"])
    if isinstance(transcription, list) and len(transcription) > 1:
        print ("more than one output received")
    else:
        transcription = transcription[0]
    
    transcription = transcription.lower()
    text = text.lower()
    
    ldist = distance(transcription, text)
    lcolor = get_lcolor(ldist)
    print (f"{gpt_cond = }")
    print (f"Original Text: {text}")
    print (f"Transcription: {transcription}")
    print (f"Levenstein: {lcolor}{ldist}{Color.F_White}")
    
    IPython.display.display(IPython.display.Audio(outputs["wav"], rate=24050, autoplay=False))

In [ ]:
num2words(27023)

### Top k

In [ ]:
top_ks = [1,3,5,7,11,15, 32,64,128,512]
for top_k in top_ks:

    text = ""
    
    test_config = copy(synth_params)
    test_config["top_k"] = top_k
    
    
    outputs = model.synthesize(
        text,
        config,
        speaker_wav="../data/trey.wav",
        language="en",
        **test_config
    )
    
    transcription = transcribe_output(outputs["wav"])
    if isinstance(transcription, list) and len(transcription) > 1:
        print ("more than one output received")
    else:
        transcription = transcription[0]
    
    transcription = transcription.lower()
    text = text.lower()
    
    ldist = distance(transcription, text)
    lcolor = get_lcolor(ldist)
    print (f"{top_k = }")
    print (f"Original Text: {text}")
    print (f"Transcription: {transcription}")
    print (f"Levenstein: {lcolor}{ldist}{Color.F_White}")
    
    IPython.display.display(IPython.display.Audio(outputs["wav"], rate=24050, autoplay=False))

### Top P

In [ ]:
top_p = [0.001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99]
for top_p in top_p:

    text = ""
    
    
    test_config = copy(synth_params)
    test_config["top_p"] = top_p
    
    outputs = model.synthesize(
        text,
        config,
        speaker_wav="../data/trey.wav",
        language="en",
        **test_config
    )
    
    transcription = transcribe_output(outputs["wav"])
    if isinstance(transcription, list) and len(transcription) > 1:
        print ("more than one output received")
    else:
        transcription = transcription[0]
    
    transcription = transcription.lower()
    text = text.lower()
    
    ldist = distance(transcription, text)
    lcolor = get_lcolor(ldist)
    print (f"{top_p = }")
    print (f"Original Text: {text}")
    print (f"Transcription: {transcription}")
    print (f"Levenstein: {lcolor}{ldist}{Color.F_White}")
    
    IPython.display.display(IPython.display.Audio(outputs["wav"], rate=24050, autoplay=False))

### Decoder Iterations

In [ ]:
decoder_iters = [1,3,5,7,11,15,64,128,512]
for decoder_iter in decoder_iters:

    text = ""
    
    test_config = copy(synth_params)
    test_config["decoder_iterations"] = decoder_iter
    
    outputs = model.synthesize(
        text,
        config,
        speaker_wav="../data/trey.wav",
        language="en",
        **test_config
    )
    
    transcription = transcribe_output(outputs["wav"])
    if isinstance(transcription, list) and len(transcription) > 1:
        print ("more than one output received")
    else:
        transcription = transcription[0]
    
    transcription = transcription.lower()
    text = text.lower()
    
    ldist = distance(transcription, text)
    lcolor = get_lcolor(ldist)
    print (f"{decoder_iter = }")
    print (f"Original Text: {text}")
    print (f"Transcription: {transcription}")
    print (f"Levenstein: {lcolor}{ldist}{Color.F_White}")
    
    IPython.display.display(IPython.display.Audio(outputs["wav"], rate=24050, autoplay=False))

### Temperature

In [ ]:
temps = [0.001, 0.005, 0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.30, 0.35, 0.40, 0.45, 0.5]
for temp in temps:

    text = ""
    
    test_config = copy(synth_params)
    test_config["temperature"] = temp
    
    outputs = model.synthesize(
        text,
        config,
        speaker_wav="../data/trey.wav",
        language="en",
        **test_config
    )
    
    transcription = transcribe_output(outputs["wav"])
    if isinstance(transcription, list) and len(transcription) > 1:
        print ("more than one output received")
    else:
        transcription = transcription[0]
    
    transcription = transcription.lower()
    text = text.lower()
    
    ldist = distance(transcription, text)
    lcolor = get_lcolor(ldist)
    print (f"{temp = }")
    print (f"Original Text: {text}")
    print (f"Transcription: {transcription}")
    print (f"Levenstein: {lcolor}{ldist}{Color.F_White}")
    
    IPython.display.display(IPython.display.Audio(outputs["wav"], rate=24050, autoplay=False))

### Repition penalty

In [ ]:
reps = [0.001, 0.01, 0.1, 0.2, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 1.9, 1.99, 2.5, 3.0, 4.0, 5.0]
for rep in reps:

    text = ""
    
    test_config = copy(synth_params)
    test_config["repetition_penalty"] = rep
    
    outputs = model.synthesize(
        text,
        config,
        speaker_wav="../data/trey.wav",
        language="en",
        **test_config
    )
    
    transcription = transcribe_output(outputs["wav"])
    if isinstance(transcription, list) and len(transcription) > 1:
        print ("more than one output received")
    else:
        transcription = transcription[0]
    
    transcription = transcription.lower()
    text = text.lower()
    
    ldist = distance(transcription, text)
    lcolor = get_lcolor(ldist)
    print (f"{rep = }")
    print (f"Original Text: {text}")
    print (f"Transcription: {transcription}")
    print (f"Levenstein: {lcolor}{ldist}{Color.F_White}")
    
    IPython.display.display(IPython.display.Audio(outputs["wav"], rate=24050, autoplay=False))

In [ ]:
synth_params

### Diffusion temperature

In [ ]:
temps = [0.001, 0.005, 0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.35, 0.5, 0.6, 0.75, 0.85, 0.95, 0.99]
for temp in temps:

    text = ""
    
    test_config = copy(synth_params)
    test_config["diffusion_temperature"] = temp
    
    outputs = model.synthesize(
        text,
        config,
        speaker_wav="../data/trey.wav",
        language="en",
        **test_config
    )
    
    transcription = transcribe_output(outputs["wav"])
    if isinstance(transcription, list) and len(transcription) > 1:
        print ("more than one output received")
    else:
        transcription = transcription[0]
    
    transcription = transcription.lower()
    text = text.lower()
    
    ldist = distance(transcription, text)
    lcolor = get_lcolor(ldist)
    print (f"{temp = }")
    print (f"Original Text: {text}")
    print (f"Transcription: {transcription}")
    print (f"Levenstein: {lcolor}{ldist}{Color.F_White}")
    
    IPython.display.display(IPython.display.Audio(outputs["wav"], rate=24050, autoplay=False))

### Cond free k
cond_free_k default 2.0

In [ ]:
ks = [0.1, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0]
for k in ks:

    text = ""
    
    test_config = copy(synth_params)
    test_config["cond_free_k"] = k
    
    outputs = model.synthesize(
        text,
        config,
        speaker_wav="../data/trey.wav",
        language="en",
        **test_config
    )
    
    wav = outputs["wav"]
    
    sf.write("cache/test.wav", wav, 24000)
    sound = AudioSegment.from_file("cache/test.wav", format = "wav")
    sound = sound.strip_silence(silence_len=250, silence_thresh=-50, padding=100)
    wav = np.array(sound.get_array_of_samples())
    
    transcription = transcribe_output(wav)
    if isinstance(transcription, list) and len(transcription) > 1:
        print ("more than one output received")
    else:
        transcription = transcription[0]
    
    transcription = transcription.lower()
    text = text.lower()
    
    ldist = distance(transcription, text)
    lcolor = get_lcolor(ldist)
    print (f"{k = }")
    print (f"Original Text: {text}")
    print (f"Transcription: {transcription}")
    print (f"Levenstein: {lcolor}{ldist}{Color.F_White}")
    
    
    IPython.display.display(IPython.display.Audio(outputs["wav"], rate=24050, autoplay=False))
    plt.plot(outputs["wav"])
    plt.show()
    IPython.display.display(IPython.display.Audio(wav, rate=24050, autoplay=False))
    plt.plot(wav)
    plt.show()

# Remove silence

In [ ]:
def detect_leading_silence(sound, silence_threshold=-50.0, chunk_size=10):
    '''
    sound is a pydub.AudioSegment
    silence_threshold in dB
    chunk_size in ms

    iterate over chunks until you find the first one with sound
    '''
    trim_ms = 0 # ms

    assert chunk_size > 0 # to avoid infinite loop
    while sound[trim_ms:trim_ms+chunk_size].dBFS < silence_threshold and trim_ms < len(sound):
        trim_ms += chunk_size

    return trim_ms


In [ ]:

start_trim = detect_leading_silence(wav)
end_trim = detect_leading_silence(wav.reverse())

duration = len(sound)    
trimmed_sound = sound[start_trim:duration-end_trim]

In [ ]:
from pydub import AudioSegment
import numpy as np

In [ ]:
#AudioSegment(wav)

In [ ]:
import matplotlib.pyplot as plt
#plt.style.use(["seaborn-darkgrid"])
plt.rcParams["figure.figsize"] = (20, 6)
plt.rcParams["figure.facecolor"] = "black"
plt.rcParams["axes.facecolor"] = "black"
plt.rcParams["axes.edgecolor"] = "white"
plt.rcParams["xtick.color"] = "white"
plt.rcParams["ytick.color"] = "white"


In [ ]:
wav = outputs["wav"]

In [ ]:

sf.write("cache/test.wav", wav, 24000)

In [ ]:
plt.plot(wav)
plt.show()

In [ ]:
IPython.display.display(IPython.display.Audio(wav, rate=24000, autoplay=False))

In [ ]:
sf.write("cache/test.wav", wav, 24000)
sound = AudioSegment.from_file("cache/test.wav", format = "wav")
sound = sound.strip_silence(silence_len=250, silence_thresh=-50, padding=100)
wav = np.array(sound.get_array_of_samples())

In [ ]:
plt.plot(wav)
plt.show()

In [ ]:
IPython.display.display(IPython.display.Audio(wav, rate=24000, autoplay=False))

In [ ]:
sound

In [ ]:
sound.strip_silence

In [ ]:
sound.strip_silence(silence_len=1000, silence_thresh=-50, padding=100)

In [ ]:
IPython.display.display(IPython.display.Audio(sound.get_array_of_samples(), rate=24000, autoplay=False))

In [ ]:
len(audio_chunks)

In [ ]:
audio_chunks[0]

In [ ]:
audio_chunks[1]

In [ ]:
type(audio_chunks[0])

In [ ]:
audio_chunks[0].speedup(1.2)

In [ ]:
audio_chunks[0].

In [ ]:
type(sound)

In [ ]:
import pydub

In [ ]:
wav.dtype.itemsize

In [ ]:
pydub.AudioSegment.from_numpy_array

In [ ]:
AudioSegment()

In [ ]:
audio_segment = pydub.AudioSegment(
    wav.tobytes(), 
    frame_rate=24000,
    sample_width=wav.dtype.itemsize, 
    channels=1
)
audio_chunks = split_on_silence(audio_segment
                            ,min_silence_len = 100
                            ,silence_thresh = -45
                            ,keep_silence = 50
                        )

In [ ]:
audio_chunks[0]

In [ ]:
audio_segment

In [ ]:
from pydub import AudioSegment, silence


In [ ]:
silence = silence.detect_silence(wav, min_silence_len=1000, silence_thresh=-16)


In [ ]:

from pydub.silence import split_on_silence

In [ ]:
aud = AudioSegment(audio.tobytes(),frame_rate = rate,
                     sample_width = audio.dtype.itemsize,channels = 1)

In [ ]:
audio_chunks = split_on_silence(
    aud,
    min_silence_len = 2000,
    silence_thresh = -45,
    keep_silence = 500,)


In [ ]:
audio_processed = sum(audio_chunks)
audio_processed = np.array(audio_processed.get_array_of_samples())

## Tuned sample

In [ ]:
temp_texts = [
    ''
]

In [ ]:
def get_model_output(model, text, synth_params):
    
    outputs = model.synthesize(
        text,
        config,
        speaker_wav="../data/trey.wav",
        language="en",
        **synth_params

    )
    
    wav = outputs["wav"]
    
    sf.write("cache/test.wav", wav, 24000)
    sound = AudioSegment.from_file("cache/test.wav", format = "wav")
    sound = sound.strip_silence(silence_len=250, silence_thresh=-50, padding=100)
    wav = np.array(sound.get_array_of_samples())
    
    
    transcription = transcribe_output(wav)
    if isinstance(transcription, list) and len(transcription) > 1:
        print ("more than one output received")
    else:
        transcription = transcription[0]
    
    transcription = transcription.lower()
    text = text.lower()
    
    ldist = distance(transcription, text)
    lcolor = get_lcolor(ldist)
    
    print(index)
    print (f"Original Text: {text}")
    print (f"Transcription: {transcription}")
    print (f"Levenstein: {lcolor}{ldist}{Color.F_White}")
    
    return wav, text, transcription, ldist

In [ ]:
synth_params = {
        #"gpt_cond_len" : 4096,
        "top_k" : 64, # Default 50
        "top_p" : 0.85, # Default 0.85
        "decoder_iterations" : 10,
        "temperature" : 0.1, # Default 0.85
        "length_penalty" : 1.0, # Default 1.0
        "repetition_penalty" : 1.8, # Default 2.0
        "cond_free_k" : 2.0, # Default 2.0
        "diffusion_temperature" : 1.0, # Default 1.0    
    }

In [ ]:
index = 0
wavs = []
sum_ldist = 0
for text in temp_texts:
    print ("-"*100)
    #for iteration in range(10): 
    wav, text, transcription, ldist = get_model_output(model, text, synth_params)

    IPython.display.display(IPython.display.Audio(wav, rate=24050, autoplay=False))
    
    print ("-"*100)
    wavs.append(outputs["wav"])
    index += 1
    sum_ldist += ldist

In [ ]:
sum_ldist

In [ ]:

from pydub.silence import split_on_silence